# Whosampled Scraper; Obtaining video IDs from breakdown pages
This post is a continuation of the 'live programming' feature developments  
to samplify.

The goal of this notebook is to illustrate the process of investigating the  
retrieval of youtube video IDs from the breakdown pages that detail the    
relationship between two songs. This page usually contains a youtube   
embed, and timings for the occurances of samples in the respective songs.

An example of this breakdown page is [Halftime -\[ contains_sample_of \]-> Dead Cast](https://www.whosampled.com/sample/52349/Nas-Halftime-Japanese-Hair-Cast-Dead-End/).

There are, however, exceptions to this rule that need to be considered, such  
as [Halftime, by Can You Flow?](https://www.whosampled.com/cover/602090/Can-You-Flow%3F-Halftime-Nas-Halftime/), which instead embeds a spotify widget.

The track we will be using to build out this functionality is [Halftime, by Nas](https://www.whosampled.com/Nas/Halftime/).  
There is a great range of content to explore on this track, and it hits all the  
direction bases. The end-to-end tests will use the following to run a lifecycle:
``` python
result = scraper.get_whosampled_playlist(
    [{"track": "halftime", "artist": ["nas"]}],
    direction=direction.all_directions
)
```

In [81]:
import sys
sys.path.append('..') # make project root code available
import json

from tools.whosampled import Scraper
from tools import direction

scraper = Scraper(verbosity=0)

def verify():
    result = scraper.get_whosampled_playlist(
        [{"track": "halftime", "artist": ["nas"]}],
        direction=direction.all_directions
    )

    print(json.dumps(result, indent=2))
    print(json.dumps(result[0]['samples'][direction.was_sampled_in], indent=2))
    print(len(result[0]['samples'][direction.was_sampled_in]))
    print(json.dumps(result, indent=2))
    

verbosity  0
verbosity == 0
verbosity  0
log_file == scraper_track.1569761090.317856


# `source` / `dest`; The ol' switcheroo
In the process of fleshing this functionality out, and browsing breakdown  
pages, it occurred to me that the use of `dest` and `source` in the breakdown  
pages is non-trivial.

In order to refer to the intended targets agnostic of direction, we will  
henceforth call the song originating in `source_playlist`, the `originator`,  
and the discovered songs, `samples`.

The `dest` / `source` naming convention reveals some business logic about the consideration  
of a 'regular' journey to this page:
- from `contains sample of`, we would see the track *containing* the sample  
  as `dest` (Halftime) and the sampled track as `source` (Dead End)
  * e.g [Halftime ->\( contains_sample_of \)-> Dead End](https://www.whosampled.com/sample/52349/Nas-Halftime-Japanese-Hair-Cast-Dead-End/)
      
- from all other directions (`was_sampled_in`, `was_remixed_in`, `was_covered_in`)  
  this is reversed, with the `originator` as `source`, and the `sample` as `dest`.  
  
It is well worth normalising this into a consistent format for use in the   
program, so the below function is a simple way to make that happen.

This will later be ported to `tools.direction`, and referenced in the code as   `direct.get_originator_from_direction`

In [82]:
import tools.direction as direct
# TODO: Put this func in tools.direction

def get_originator_from_direction(direction, breakdown_ids):
    """ normalise 'dest' / 'source' relationship as 'originator' / 'sample',
        where the originator is the subject of the initial request
        - this assumes the return from whosampled.get_videos_from_breakdown
          is [DEST, SOURCE]
    """
    print('ids')
    print(breakdown_ids)
    
    if direction == direct.contains_sample_of:
        return breakdown_ids[0], breakdown_ids[1] # originator is dest
    
    if direction == direct.was_sampled_in or \
       direction == direct.was_remixed_in or \
       direction == direct.was_covered_in:
        return breakdown_ids[1], breakdown_ids[0] # originator is source
    
    raise Exception(f'Invalid direction: {direction}')

# Get videos from Breakdown
This turned out to be a very simple task. The requests boilerplate is plumbed  
in as we seemingly cannot access `self` from a stubbed function - this is most  
certainly a failing on my part somewhere.

As we can rely on the sorting from `direct.get_originator_from_direction`, we can  
refine the soup to the video containers, then list comp out the properties we need.

Thankfully, each the video ID can be found in the `id` attribute of the iframe, so  
extracting it is as simple as:
``` python
IFRAME.get('id')
```

In [83]:
import requests
from bs4 import BeautifulSoup

def get_videos_from_breakdown(link):
    # S: boilerplate (replace with call to self.req when refactored)
    base_url = 'https://whosampled.com'
    req = requests.Session()
    adapter = requests.adapters.HTTPAdapter(max_retries=10)
    req.mount('https://', adapter)
    req.mount('http://', adapter)
    req.headers = {
        "User-Agent":
        "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"
    }
    # E: boilerplate (replace with call to self.req when refactored)
    
    breakdown = BeautifulSoup(req.get(base_url + link).content)
    ids = [div.iframe.get('id') for div in breakdown.find_all('div', attrs={'class': 'sample-embed'})]
    return ids # subject is normalised above as (originator, sample)

# Extending `parse_sample_items`
Let us continue by stubbing `parse_sample_items`, and building out the functionality  
we're looking for. This is also a great opportunity to replace the implicit property  
retrieval based on tab / newline delimitation we've been using until now.

We will be piecing this functionality into tools.whosampled, around the existing  
functionality in `parse_sample_items`. Once the video IDs are available for use in  
`sample_finder.py`, it will be time to decouple spotify from the core sorting and  
processing of the application, and add sets of mappings for new platforms (youtube  
being the obvious next step).

In [84]:
def stub__parse_sample_items(sample_data, direction, link, recursing_page=False):
    # rip boilerplate
    find_class = lambda s,e,c: s.find(e, attrs={'class': c})
    
    raw_samples = sample_data.findAll('div', attrs={'class': 'sampleEntry'})
    # print(raw_samples)
    parsed_samples = []
    
    for sample in raw_samples:
        track_artist = find_class(sample, 'span','trackArtist')
        track_name = find_class(sample, 'a', 'trackName')
        track_badge = find_class(sample, 'div', 'trackBadge')
        track_type = find_class(track_badge, 'span', 'topItem')
        track_genre = find_class(track_badge, 'span', 'bottomItem')
        breakdown_ref = track_name.get('href')
        print(breakdown_ref)
        ids = get_videos_from_breakdown(breakdown_ref) # remember to selfify this once refactored
        yt_originator, yt_sample = get_originator_from_direction(direction, ids)
        parsed_samples.append({
            'type': track_type.text if track_type else '',
            'genre': track_genre.text if track_genre else '',
            'title': track_name.text,
            'artist': [a.text for a in track_artist.find_all('a')],
            'year': track_artist.text[-9:-5],
            'yt_breakdown': track_name.get('href'),
            'yt_originator': yt_originator,
            'yt_sample': yt_sample
        })
    print(parsed_samples)
    return parsed_samples

scraper.parse_sample_items = stub__parse_sample_items

# Verification
- unit tests
- all direction groups hit
- no recursion
- counts of samples

```json
verbosity  0
Message: Scraping for source_playlist
verbosity  0
Message: Getting page: https://whosampled.com/Nas/Halftime/
/sample/52349/Nas-Halftime-Japanese-Hair-Cast-Dead-End/
ids
['UG7lhaAo4CM', 'VIjpDv8CySI']
/sample/976/Nas-Halftime-Average-White-Band-School-Boy-Crush/
ids
['UG7lhaAo4CM', '6qnB-O76xv0']
/sample/17077/Nas-Halftime-Gary-Byrd-Soul-Travelin%27-Pt.-I-(The-G.B.E.)/
ids
['UG7lhaAo4CM', 'D4OuBGoZAsA']
[{'type': 'Bass', 'genre': 'Soundtrack', 'title': 'Dead End', 'artist': ['Japanese Hair Cast'], 'year': '1971', 'yt_breakdown': '/sample/52349/Nas-Halftime-Japanese-Hair-Cast-Dead-End/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': 'VIjpDv8CySI'}, {'type': 'Multiple Elements', 'genre': 'Soul / Funk / Disco', 'title': 'School Boy Crush', 'artist': ['Average White Band'], 'year': '1975', 'yt_breakdown': '/sample/976/Nas-Halftime-Average-White-Band-School-Boy-Crush/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': '6qnB-O76xv0'}, {'type': 'Hook / Riff', 'genre': 'Soul / Funk / Disco', 'title': "Soul Travelin' Pt. I (The G.B.E.)", 'artist': ['Gary Byrd'], 'year': '1973', 'yt_breakdown': '/sample/17077/Nas-Halftime-Gary-Byrd-Soul-Travelin%27-Pt.-I-(The-G.B.E.)/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': 'D4OuBGoZAsA'}]
/sample/22236/Nas-The-Message-Nas-Halftime/
ids
['8FlNIcqApLg', 'UG7lhaAo4CM']
/sample/603307/Black-Eyed-Peas-Nas-BACK-2-HIPHOP-Nas-Halftime/
ids
['T3um72hrtrk', 'UG7lhaAo4CM']
/sample/576718/Pawz-One-Robin-Da-Landlord-It%27s-Like-That-(Remix)-Nas-Halftime/
ids
[None, 'UG7lhaAo4CM']
/sample/16065/Army-of-the-Pharaohs-Strike-Back-Nas-Halftime/
ids
['M3Abr_gK3LQ', 'UG7lhaAo4CM']
/sample/158818/G-EAZY-Reefer-Madness-Nas-Halftime/
ids
['AteYGkaws3U', 'UG7lhaAo4CM']
[{'type': 'Vocals / Lyrics', 'genre': 'Hip-Hop / Rap / R&B', 'title': 'The Message', 'artist': ['Nas'], 'year': '1996', 'yt_breakdown': '/sample/22236/Nas-The-Message-Nas-Halftime/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': '8FlNIcqApLg'}, {'type': 'Vocals / Lyrics', 'genre': 'Hip-Hop / Rap / R&B', 'title': 'BACK 2 HIPHOP', 'artist': ['Black Eyed Peas', 'Nas'], 'year': '2018', 'yt_breakdown': '/sample/603307/Black-Eyed-Peas-Nas-BACK-2-HIPHOP-Nas-Halftime/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': 'T3um72hrtrk'}, {'type': 'Vocals / Lyrics', 'genre': 'Hip-Hop / Rap / R&B', 'title': "It's Like That (Remix)", 'artist': ['Pawz One', 'Robin Da Landlord'], 'year': '2018', 'yt_breakdown': '/sample/576718/Pawz-One-Robin-Da-Landlord-It%27s-Like-That-(Remix)-Nas-Halftime/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': None}, {'type': 'Vocals / Lyrics', 'genre': 'Hip-Hop / Rap / R&B', 'title': 'Strike Back', 'artist': ['Army of the Pharaohs'], 'year': '2007', 'yt_breakdown': '/sample/16065/Army-of-the-Pharaohs-Strike-Back-Nas-Halftime/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': 'M3Abr_gK3LQ'}, {'type': 'Vocals / Lyrics', 'genre': 'Hip-Hop / Rap / R&B', 'title': 'Reefer Madness', 'artist': ['G-EAZY'], 'year': '2011', 'yt_breakdown': '/sample/158818/G-EAZY-Reefer-Madness-Nas-Halftime/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': 'AteYGkaws3U'}]
/remix/181488/Nas-Halftime-(Butcher-Remix)-Nas-Halftime/
ids
['Qbg12RBab9g', 'UG7lhaAo4CM']
[{'type': 'Hip-Hop / Rap / R&B', 'genre': '', 'title': 'Halftime (Butcher Remix)', 'artist': ['Joe "The Butcher" Nicolo'], 'year': '1992', 'yt_breakdown': '/remix/181488/Nas-Halftime-(Butcher-Remix)-Nas-Halftime/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': 'Qbg12RBab9g'}]
/cover/66888/Fashawn-Halftime-Nas-Halftime/
ids
['Pzol0Oy5SYU', 'UG7lhaAo4CM']
/cover/148766/Elzhi-Will-Sessions-Halftime-Nas-Halftime/
ids
['vcKzOZ6Yc78', 'UG7lhaAo4CM']
/cover/602090/Can-You-Flow%3F-Halftime-Nas-Halftime/
ids
[None, 'UG7lhaAo4CM']
[{'type': 'Hip-Hop / Rap / R&B', 'genre': '', 'title': 'Halftime', 'artist': ['Fashawn'], 'year': '2010', 'yt_breakdown': '/cover/66888/Fashawn-Halftime-Nas-Halftime/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': 'Pzol0Oy5SYU'}, {'type': 'Hip-Hop / Rap / R&B', 'genre': '', 'title': 'Halftime', 'artist': ['Elzhi', 'Will Sessions'], 'year': '2011', 'yt_breakdown': '/cover/148766/Elzhi-Will-Sessions-Halftime-Nas-Halftime/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': 'vcKzOZ6Yc78'}, {'type': 'Hip-Hop / Rap / R&B', 'genre': '', 'title': 'Halftime', 'artist': ['Can You Flow?'], 'year': '2006', 'yt_breakdown': '/cover/602090/Can-You-Flow%3F-Halftime-Nas-Halftime/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': None}]
[
  {
    "track": "halftime",
    "artist": [
      "nas"
    ],
    "samples": {
      "Contains samples of": [
        {
          "type": "Bass",
          "genre": "Soundtrack",
          "title": "Dead End",
          "artist": [
            "Japanese Hair Cast"
          ],
          "year": "1971",
          "yt_breakdown": "/sample/52349/Nas-Halftime-Japanese-Hair-Cast-Dead-End/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "VIjpDv8CySI"
        },
        {
          "type": "Multiple Elements",
          "genre": "Soul / Funk / Disco",
          "title": "School Boy Crush",
          "artist": [
            "Average White Band"
          ],
          "year": "1975",
          "yt_breakdown": "/sample/976/Nas-Halftime-Average-White-Band-School-Boy-Crush/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "6qnB-O76xv0"
        },
        {
          "type": "Hook / Riff",
          "genre": "Soul / Funk / Disco",
          "title": "Soul Travelin' Pt. I (The G.B.E.)",
          "artist": [
            "Gary Byrd"
          ],
          "year": "1973",
          "yt_breakdown": "/sample/17077/Nas-Halftime-Gary-Byrd-Soul-Travelin%27-Pt.-I-(The-G.B.E.)/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "D4OuBGoZAsA"
        }
      ],
      "Was sampled in": [
        {
          "type": "Vocals / Lyrics",
          "genre": "Hip-Hop / Rap / R&B",
          "title": "The Message",
          "artist": [
            "Nas"
          ],
          "year": "1996",
          "yt_breakdown": "/sample/22236/Nas-The-Message-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "8FlNIcqApLg"
        },
        {
          "type": "Vocals / Lyrics",
          "genre": "Hip-Hop / Rap / R&B",
          "title": "BACK 2 HIPHOP",
          "artist": [
            "Black Eyed Peas",
            "Nas"
          ],
          "year": "2018",
          "yt_breakdown": "/sample/603307/Black-Eyed-Peas-Nas-BACK-2-HIPHOP-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "T3um72hrtrk"
        },
        {
          "type": "Vocals / Lyrics",
          "genre": "Hip-Hop / Rap / R&B",
          "title": "It's Like That (Remix)",
          "artist": [
            "Pawz One",
            "Robin Da Landlord"
          ],
          "year": "2018",
          "yt_breakdown": "/sample/576718/Pawz-One-Robin-Da-Landlord-It%27s-Like-That-(Remix)-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": null
        },
        {
          "type": "Vocals / Lyrics",
          "genre": "Hip-Hop / Rap / R&B",
          "title": "Strike Back",
          "artist": [
            "Army of the Pharaohs"
          ],
          "year": "2007",
          "yt_breakdown": "/sample/16065/Army-of-the-Pharaohs-Strike-Back-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "M3Abr_gK3LQ"
        },
        {
          "type": "Vocals / Lyrics",
          "genre": "Hip-Hop / Rap / R&B",
          "title": "Reefer Madness",
          "artist": [
            "G-EAZY"
          ],
          "year": "2011",
          "yt_breakdown": "/sample/158818/G-EAZY-Reefer-Madness-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "AteYGkaws3U"
        }
      ],
      "Was remixed in": [
        {
          "type": "Hip-Hop / Rap / R&B",
          "genre": "",
          "title": "Halftime (Butcher Remix)",
          "artist": [
            "Joe \"The Butcher\" Nicolo"
          ],
          "year": "1992",
          "yt_breakdown": "/remix/181488/Nas-Halftime-(Butcher-Remix)-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "Qbg12RBab9g"
        }
      ],
      "Was covered in": [
        {
          "type": "Hip-Hop / Rap / R&B",
          "genre": "",
          "title": "Halftime",
          "artist": [
            "Fashawn"
          ],
          "year": "2010",
          "yt_breakdown": "/cover/66888/Fashawn-Halftime-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "Pzol0Oy5SYU"
        },
        {
          "type": "Hip-Hop / Rap / R&B",
          "genre": "",
          "title": "Halftime",
          "artist": [
            "Elzhi",
            "Will Sessions"
          ],
          "year": "2011",
          "yt_breakdown": "/cover/148766/Elzhi-Will-Sessions-Halftime-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "vcKzOZ6Yc78"
        },
        {
          "type": "Hip-Hop / Rap / R&B",
          "genre": "",
          "title": "Halftime",
          "artist": [
            "Can You Flow?"
          ],
          "year": "2006",
          "yt_breakdown": "/cover/602090/Can-You-Flow%3F-Halftime-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": null
        }
      ]
    }
  }
]
[
  {
    "type": "Vocals / Lyrics",
    "genre": "Hip-Hop / Rap / R&B",
    "title": "The Message",
    "artist": [
      "Nas"
    ],
    "year": "1996",
    "yt_breakdown": "/sample/22236/Nas-The-Message-Nas-Halftime/",
    "yt_originator": "UG7lhaAo4CM",
    "yt_sample": "8FlNIcqApLg"
  },
  {
    "type": "Vocals / Lyrics",
    "genre": "Hip-Hop / Rap / R&B",
    "title": "BACK 2 HIPHOP",
    "artist": [
      "Black Eyed Peas",
      "Nas"
    ],
    "year": "2018",
    "yt_breakdown": "/sample/603307/Black-Eyed-Peas-Nas-BACK-2-HIPHOP-Nas-Halftime/",
    "yt_originator": "UG7lhaAo4CM",
    "yt_sample": "T3um72hrtrk"
  },
  {
    "type": "Vocals / Lyrics",
    "genre": "Hip-Hop / Rap / R&B",
    "title": "It's Like That (Remix)",
    "artist": [
      "Pawz One",
      "Robin Da Landlord"
    ],
    "year": "2018",
    "yt_breakdown": "/sample/576718/Pawz-One-Robin-Da-Landlord-It%27s-Like-That-(Remix)-Nas-Halftime/",
    "yt_originator": "UG7lhaAo4CM",
    "yt_sample": null
  },
  {
    "type": "Vocals / Lyrics",
    "genre": "Hip-Hop / Rap / R&B",
    "title": "Strike Back",
    "artist": [
      "Army of the Pharaohs"
    ],
    "year": "2007",
    "yt_breakdown": "/sample/16065/Army-of-the-Pharaohs-Strike-Back-Nas-Halftime/",
    "yt_originator": "UG7lhaAo4CM",
    "yt_sample": "M3Abr_gK3LQ"
  },
  {
    "type": "Vocals / Lyrics",
    "genre": "Hip-Hop / Rap / R&B",
    "title": "Reefer Madness",
    "artist": [
      "G-EAZY"
    ],
    "year": "2011",
    "yt_breakdown": "/sample/158818/G-EAZY-Reefer-Madness-Nas-Halftime/",
    "yt_originator": "UG7lhaAo4CM",
    "yt_sample": "AteYGkaws3U"
  }
]
5
[
  {
    "track": "halftime",
    "artist": [
      "nas"
    ],
    "samples": {
      "Contains samples of": [
        {
          "type": "Bass",
          "genre": "Soundtrack",
          "title": "Dead End",
          "artist": [
            "Japanese Hair Cast"
          ],
          "year": "1971",
          "yt_breakdown": "/sample/52349/Nas-Halftime-Japanese-Hair-Cast-Dead-End/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "VIjpDv8CySI"
        },
        {
          "type": "Multiple Elements",
          "genre": "Soul / Funk / Disco",
          "title": "School Boy Crush",
          "artist": [
            "Average White Band"
          ],
          "year": "1975",
          "yt_breakdown": "/sample/976/Nas-Halftime-Average-White-Band-School-Boy-Crush/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "6qnB-O76xv0"
        },
        {
          "type": "Hook / Riff",
          "genre": "Soul / Funk / Disco",
          "title": "Soul Travelin' Pt. I (The G.B.E.)",
          "artist": [
            "Gary Byrd"
          ],
          "year": "1973",
          "yt_breakdown": "/sample/17077/Nas-Halftime-Gary-Byrd-Soul-Travelin%27-Pt.-I-(The-G.B.E.)/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "D4OuBGoZAsA"
        }
      ],
      "Was sampled in": [
        {
          "type": "Vocals / Lyrics",
          "genre": "Hip-Hop / Rap / R&B",
          "title": "The Message",
          "artist": [
            "Nas"
          ],
          "year": "1996",
          "yt_breakdown": "/sample/22236/Nas-The-Message-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "8FlNIcqApLg"
        },
        {
          "type": "Vocals / Lyrics",
          "genre": "Hip-Hop / Rap / R&B",
          "title": "BACK 2 HIPHOP",
          "artist": [
            "Black Eyed Peas",
            "Nas"
          ],
          "year": "2018",
          "yt_breakdown": "/sample/603307/Black-Eyed-Peas-Nas-BACK-2-HIPHOP-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "T3um72hrtrk"
        },
        {
          "type": "Vocals / Lyrics",
          "genre": "Hip-Hop / Rap / R&B",
          "title": "It's Like That (Remix)",
          "artist": [
            "Pawz One",
            "Robin Da Landlord"
          ],
          "year": "2018",
          "yt_breakdown": "/sample/576718/Pawz-One-Robin-Da-Landlord-It%27s-Like-That-(Remix)-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": null
        },
        {
          "type": "Vocals / Lyrics",
          "genre": "Hip-Hop / Rap / R&B",
          "title": "Strike Back",
          "artist": [
            "Army of the Pharaohs"
          ],
          "year": "2007",
          "yt_breakdown": "/sample/16065/Army-of-the-Pharaohs-Strike-Back-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "M3Abr_gK3LQ"
        },
        {
          "type": "Vocals / Lyrics",
          "genre": "Hip-Hop / Rap / R&B",
          "title": "Reefer Madness",
          "artist": [
            "G-EAZY"
          ],
          "year": "2011",
          "yt_breakdown": "/sample/158818/G-EAZY-Reefer-Madness-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "AteYGkaws3U"
        }
      ],
      "Was remixed in": [
        {
          "type": "Hip-Hop / Rap / R&B",
          "genre": "",
          "title": "Halftime (Butcher Remix)",
          "artist": [
            "Joe \"The Butcher\" Nicolo"
          ],
          "year": "1992",
          "yt_breakdown": "/remix/181488/Nas-Halftime-(Butcher-Remix)-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "Qbg12RBab9g"
        }
      ],
      "Was covered in": [
        {
          "type": "Hip-Hop / Rap / R&B",
          "genre": "",
          "title": "Halftime",
          "artist": [
            "Fashawn"
          ],
          "year": "2010",
          "yt_breakdown": "/cover/66888/Fashawn-Halftime-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "Pzol0Oy5SYU"
        },
        {
          "type": "Hip-Hop / Rap / R&B",
          "genre": "",
          "title": "Halftime",
          "artist": [
            "Elzhi",
            "Will Sessions"
          ],
          "year": "2011",
          "yt_breakdown": "/cover/148766/Elzhi-Will-Sessions-Halftime-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": "vcKzOZ6Yc78"
        },
        {
          "type": "Hip-Hop / Rap / R&B",
          "genre": "",
          "title": "Halftime",
          "artist": [
            "Can You Flow?"
          ],
          "year": "2006",
          "yt_breakdown": "/cover/602090/Can-You-Flow%3F-Halftime-Nas-Halftime/",
          "yt_originator": "UG7lhaAo4CM",
          "yt_sample": null
        }
      ]
    }
  }
]
```

In [85]:
verify()

verbosity  0
Message: Scraping for source_playlist
verbosity  0
Message: Getting page: https://whosampled.com/Nas/Halftime/
/sample/52349/Nas-Halftime-Japanese-Hair-Cast-Dead-End/
ids
['UG7lhaAo4CM', 'VIjpDv8CySI']
/sample/976/Nas-Halftime-Average-White-Band-School-Boy-Crush/
ids
['UG7lhaAo4CM', '6qnB-O76xv0']
/sample/17077/Nas-Halftime-Gary-Byrd-Soul-Travelin%27-Pt.-I-(The-G.B.E.)/
ids
['UG7lhaAo4CM', 'D4OuBGoZAsA']
[{'type': 'Bass', 'genre': 'Soundtrack', 'title': 'Dead End', 'artist': ['Japanese Hair Cast'], 'year': '1971', 'yt_breakdown': '/sample/52349/Nas-Halftime-Japanese-Hair-Cast-Dead-End/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': 'VIjpDv8CySI'}, {'type': 'Multiple Elements', 'genre': 'Soul / Funk / Disco', 'title': 'School Boy Crush', 'artist': ['Average White Band'], 'year': '1975', 'yt_breakdown': '/sample/976/Nas-Halftime-Average-White-Band-School-Boy-Crush/', 'yt_originator': 'UG7lhaAo4CM', 'yt_sample': '6qnB-O76xv0'}, {'type': 'Hook / Riff', 'genre': 'Soul / Funk / 